# Imports and Environment Variables

In [19]:
from ccip_terminal import (network_func, get_ccip_fee_estimate, approve_token_if_needed,
                           send_ccip_transfer, check_ccip_message_status, get_usdc_data,
                           ROUTER_MAP,get_largest_balance)
import json
from web3 import Web3
import os
from dotenv import load_dotenv

ImportError: cannot import name 'get_largest_balance' from 'ccip_terminal' (E:\Projects\ccip_terminal_copy\ccip_terminal\__init__.py)

In [ ]:
load_dotenv()

True

: 

In [ ]:
PRIVATE_KEY = os.getenv('PRIVATE_KEY')

: 

In [ ]:
network = 'arbitrum' # Can set to variety of networks

w3 = network_func(network)
account = w3.eth.account.from_key(PRIVATE_KEY)
account_obj={"account": account, "w3": w3}

: 

# Sending a CCIP Transfer

In [ ]:
BALANCES_DICT, TOKEN_CONTRACTS, TOKEN_DECIMALS, _, usdc_price = get_usdc_data(account_obj=account_obj)
# USDC Balance for Account Across Supported Networks

print(json.dumps(BALANCES_DICT, indent=2))

{
  "0x38979DFdB5d8FD76FAD4E797c4660e20015C6a84": {
    "ethereum": {
      "usdc": 0.00979,
      "native_token": 4.132747937321e-06
    },
    "optimism": {
      "usdc": 1.0,
      "native_token": 0.001397223622087085
    },
    "avalanche": {
      "usdc": 1.0,
      "native_token": 0.0
    },
    "arbitrum": {
      "usdc": 0.454262,
      "native_token": 0.002657681112361215
    },
    "polygon": {
      "usdc": 1.0,
      "native_token": 0.0
    },
    "base": {
      "usdc": 5.0,
      "native_token": 0.0
    }
  }
}


: 

In [ ]:
largest_balance_dict = get_largest_balance(BALANCES_DICT, account_obj=account_obj,min_gas_threshold=0.003)

In [6]:
to_address = '0x2083b0413869f7b5b9e0eA27d20CB6dD3535f525'
dest_chain='base'
amount=0.4
source_chain=network

In [7]:
TOKEN_CONTRACTS

{'ethereum': '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
 'optimism': '0x0b2C639c533813f4Aa9D7837CAf62653d097Ff85',
 'avalanche': '0xB97EF9Ef8734C71904D8002F8b6Bc66Dd9c48a6E',
 'arbitrum': '0xaf88d065e77c8cC2239327C5EDb3A432268e5831',
 'polygon': '0x3c499c542cEF5E3811e1192ce70d8cC03d5c3359',
 'base': '0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913'}

In [10]:
source_chain_state = BALANCES_DICT[account.address][source_chain]
source_chain_state['usdc']
usdc_balance = source_chain_state['usdc']
sufficient_token = amount <= usdc_balance

if not sufficient_token:
    print(f'Token balance below desired send amount - Account: {usdc_balance}, Desired amount: {amount}')
else:
    print(f'Token balance is sufficient for send amount')

Token balance is sufficient for send amount


In [11]:
# We need to use custom fee estimation function for CCIP

estimate = get_ccip_fee_estimate(
    to_address=to_address,
    dest_chain=dest_chain,
    amount=amount,
    source_chain=source_chain,
    account_obj=account_obj
)
native_balance = w3.eth.get_balance(account.address)
total_estimate = estimate["total_estimate"]

print(f'native gas balance: {native_balance / 1e18}')
print(f'gas estimate: {total_estimate / 1e18}')

can_transfer = native_balance > total_estimate

print(f'Enough gas for transfer?: {"yes" if can_transfer else "no"}')

Getting token data
native gas balance: 0.002657681112361215
gas estimate: 0.002210126506080179
Enough gas for transfer?: yes


In [12]:
# Note fee is overestimated as we cannot directly estimate gas. For L2 transfers, fee typically ~0.00018 or $0.24

estimate # Denominated in wei

{'total_estimate': 2210126506080179,
 'native_gas_fee': 2034054990000000,
 'ccip_fee': 176071516080179,
 'gas_limit': 405999,
 'max_fee_per_gas': 5010000000,
 'priority_fee': 5000000000,
 'base_gas_price': 10000000,
 'currency': 42161}

In [13]:
#Approves Router to Spend Our Token If Necessary

approve_token_if_needed(TOKEN_CONTRACTS[source_chain], 
                            ROUTER_MAP[source_chain], 
                            account_obj, 
                            threshold=None)

True

In [14]:
receipt = None

if can_transfer and sufficient_token:
    receipt, links, success, message_id = send_ccip_transfer(
        to_address=to_address,
        dest_chain=dest_chain,
        amount=amount,
        source_chain=source_chain,
        account_obj=account_obj,
        estimate=estimate["total_estimate"]
    )
else:
    missing_gas = estimate["total_estimate"] - native_balance
    missing_token = amount - usdc_balance 
    print(f'Need to top up on gas; need {missing_gas / 1e18} more native token')
    print(f'May need more USDC in account or need to lower desired send amount: {missing_token}')


2025-04-09 17:53:47,942 - INFO - CCIP messageId (pre-send): 6c9ef508f8e06185652d2146e2478fe03b2b000205e79c5e5ba7b60a4fad771d


account.address: 0x38979DFdB5d8FD76FAD4E797c4660e20015C6a84


In [18]:
if receipt and receipt.status == 1:
    print(f'Tx Links:{links}')
    print(f'CCIP MessageID: {message_id}')
    # Here we query Etherscan for transfer status, can take up to 25 minutes.
    # Wait flag periodically calls the API for status
    check_ccip_message_status(message_id, dest_chain, wait=True)
else:
    print(F'No Tx Detected or Tx Failed')

Tx Links:{'source_url': 'https://arbitrum.blockscout.com/tx/0xe33a475176163b74c733b8e3ec5cf902527a5a66661a98240d545a64ecbc6bde', 'ccip_url': 'https://ccip.chain.link/#/side-drawer/msg/0x6c9ef508f8e06185652d2146e2478fe03b2b000205e79c5e5ba7b60a4fad771d'}
CCIP MessageID: 6c9ef508f8e06185652d2146e2478fe03b2b000205e79c5e5ba7b60a4fad771d
Attempt 1/15 – retrying in 120s...
Attempt 2/15 – retrying in 120s...
Attempt 3/15 – retrying in 120s...
Attempt 4/15 – retrying in 120s...
Attempt 5/15 – retrying in 120s...
Attempt 6/15 – retrying in 120s...
Attempt 7/15 – retrying in 120s...
Attempt 8/15 – retrying in 120s...
Attempt 9/15 – retrying in 120s...
Attempt 10/15 – retrying in 120s...
Attempt 11/15 – retrying in 120s...
Attempt 12/15 – retrying in 120s...
Found status for message 6c9ef508f8e06185652d2146e2478fe03b2b000205e79c5e5ba7b60a4fad771d on 0x7d38c6363d5e4dfd500a691bc34878b383f58d93: SUCCESS
